In [1]:
import time
import json
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
src = "user_log_format1.csv"

In [2]:
t0 = time.clock()
df = pd.read_csv(src)
print("load data\nruntime:", time.clock() - t0)

load data
runtime: 27.550575024077926


In [3]:
buy_raw = df.loc[df["action_type"]==2]
print("all purchase:", buy_raw.shape)

buy_unique = buy_raw.drop_duplicates(['user_id', 'item_id'])
print("unique purchase:", buy_unique.shape)


all purchase: (3292144, 7)
unique purchase: (2939895, 7)


In [34]:
dataset = []
ct = 0
t0 = time.clock()
for ind, row in buy_unique.iterrows():
    dataset += [df[(df.user_id == row[0]) & (df.item_id == row[1])].to_dict('list')]
    ct += 1
    if ct == 10000:
        break
print("load data\nruntime:", time.clock() - t0)

load data
runtime: 27.247911753011522


In [35]:
with open('test.json', 'w') as f:
    json.dump(dataset, f, indent=4)

In [36]:
conn = sqlite3.connect("skycat.db")

t0 = time.clock()
query = "select distinct user_id, item_id from user_log_batch where action_type=2"
cursor = conn.execute(query)
conn.commit()
result = cursor.fetchall()
print(len(result))
print("runtime: %.4f" % (time.clock() - t0))

2939895
runtime: 20.9159


In [37]:
query = "select * from user_log_batch where user_id=? and item_id=?;"
for i, r in enumerate(result):
    cursor = conn.execute(query, r)
    conn.commit()
    subresult = cursor.fetchall()
    for sr in subresult:
        print(sr)
    if i == 0:
        break

(328862, 81766, 614, 4605, 7622, 709, 2)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 801, 2)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 801, 0)
(328862, 81766, 614, 4605, 7622, 801, 0)
(328862, 81766, 614, 4605, 7622, 709, 2)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 801, 2)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 709, 0)
(328862, 81766, 614, 4605, 7622, 801, 0)
(328862, 81766, 614, 4605, 7622, 801, 0)


In [ ]:
dst = "sliced_unique_purchase/"
buy_unique.iloc[0:1000000,:].to_csv(dst + 'p1.csv', index=False)
buy_unique.iloc[1000000:2000000,:].to_csv(dst + 'p2.csv', index=False)
buy_unique.iloc[2000000:2939895,:].to_csv(dst + 'p3.csv', index=False)
buy_unique.to_csv(dst + "all_purchase.csv", index=0)